# ILEC Data Cleaning for Two-Stage LGBM Mortality Prediction

This notebook prepares the ILEC 2012-2019 mortality dataset for two-stage LGBM modeling.

## Two-Stage Strategy
- **Stage 1**: Baseline model with 11 core features (no Year)
- **Stage 2**: Residual analysis using Observation_Year

```
Stage 1: baseline_pred = model(Age, Sex, Smoker, etc.)
         ↓
Stage 2: residual = observed - baseline → analyze by Year
```

In [10]:
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# File paths
DATA_PATH = '../data/ILEC_2012_19 - 20240429.txt'
OUTPUT_PATH = '../data/ilec_cleaned.parquet'
CHUNK_SIZE = 500000

print('Setup complete!')

Setup complete!


## 1. Define Feature Strategy

In [ ]:
# ========================================
# Stage 1 Features (11 total, NO Year)
# ========================================

NUMERICAL_FEATURES = [
    'Attained_Age',
    'Issue_Age', 
    'Duration'
]

CATEGORICAL_FEATURES = [
    'Sex',
    'Smoker_Status',
    'Insurance_Plan',
    'Face_Amount_Band',
    'Preferred_Class',      # Fill NA with 'NA'
    'SOA_Post_Lvl_Ind',     # Fill NA with 'NA'
    'SOA_Antp_Lvl_TP',
    'SOA_Guar_Lvl_TP'
]

# ========================================
# Stage 2: Year for residual analysis
# ========================================
STAGE2_FEATURES = ['Observation_Year']

# Columns to fill NA with 'NA' category
FILL_NA_COLS = ['Preferred_Class', 'SOA_Post_Lvl_Ind']

# Target and weight
TARGET = 'Death_Count'
WEIGHT = 'Policies_Exposed'

# All columns to read
STAGE1_FEATURES = NUMERICAL_FEATURES + CATEGORICAL_FEATURES
USE_COLS = STAGE1_FEATURES + STAGE2_FEATURES + [TARGET, WEIGHT]

print(f'Stage 1 Features: {len(STAGE1_FEATURES)}')
print(f'  Numerical ({len(NUMERICAL_FEATURES)}): {NUMERICAL_FEATURES}')
print(f'  Categorical ({len(CATEGORICAL_FEATURES)}): {CATEGORICAL_FEATURES}')
print(f'\nStage 2 Features: {STAGE2_FEATURES}')
print(f'\nTarget: {TARGET}')
print(f'Weight: {WEIGHT}')

Stage 1 Features: 11
  Numerical (3): ['Attained_Age', 'Issue_Age', 'Duration']
  Categorical (8): ['Sex', 'Smoker_Status', 'Insurance_Plan', 'Face_Amount_Band', 'Preferred_Class', 'SOA_Post_Lvl_Ind', 'SOA_Antp_Lvl_TP', 'SOA_Guar_Lvl_TP']

Stage 2 Features: ['Observation_Year']

Target: Death_Count
Weight: Policies_Exposed


## 2. Load and Clean Data (Chunk-based)

In [12]:
print('Starting data loading and cleaning...')
print('=' * 60)

cleaned_chunks = []
chunk_num = 0
total_rows = 0

for chunk in pd.read_csv(DATA_PATH, sep='\t', chunksize=CHUNK_SIZE, usecols=USE_COLS):
    chunk_num += 1
    total_rows += len(chunk)
    
    # Fill NA for specified columns
    for col in FILL_NA_COLS:
        chunk[col] = chunk[col].fillna('NA').astype(str)
    
    # Convert categorical columns to category dtype
    for col in CATEGORICAL_FEATURES:
        chunk[col] = chunk[col].astype('category')
    
    cleaned_chunks.append(chunk)
    
    if chunk_num % 10 == 0:
        print(f'  Processed {chunk_num} chunks ({total_rows:,} records)')

print(f'\nLoading complete! Total: {total_rows:,} records')
print('=' * 60)

Starting data loading and cleaning...
  Processed 10 chunks (5,000,000 records)
  Processed 20 chunks (10,000,000 records)
  Processed 30 chunks (15,000,000 records)
  Processed 40 chunks (20,000,000 records)
  Processed 50 chunks (25,000,000 records)
  Processed 60 chunks (30,000,000 records)
  Processed 70 chunks (35,000,000 records)
  Processed 80 chunks (40,000,000 records)
  Processed 90 chunks (45,000,000 records)

Loading complete! Total: 45,501,036 records


In [13]:
# Combine all chunks
print('Combining chunks...')
df = pd.concat(cleaned_chunks, ignore_index=True)
del cleaned_chunks  # Free memory

print(f'Combined DataFrame shape: {df.shape}')
print(f'Memory usage: {df.memory_usage(deep=True).sum() / 1e9:.2f} GB')

Combining chunks...
Combined DataFrame shape: (45501036, 14)
Memory usage: 22.75 GB


## 3. Data Validation

In [14]:
print('=== Data Validation ===')
print(f'\nShape: {df.shape}')
print(f'\nColumn dtypes:')
print(df.dtypes)
print(f'\nMissing values:')
print(df.isnull().sum())

=== Data Validation ===

Shape: (45501036, 14)

Column dtypes:
Observation_Year      int64
Sex                  object
Smoker_Status        object
Insurance_Plan       object
Issue_Age             int64
Duration              int64
Face_Amount_Band     object
Attained_Age          int64
SOA_Antp_Lvl_TP      object
SOA_Guar_Lvl_TP      object
SOA_Post_Lvl_Ind     object
Preferred_Class      object
Policies_Exposed    float64
Death_Count           int64
dtype: object

Missing values:
Observation_Year    0
Sex                 0
Smoker_Status       0
Insurance_Plan      0
Issue_Age           0
Duration            0
Face_Amount_Band    0
Attained_Age        0
SOA_Antp_Lvl_TP     0
SOA_Guar_Lvl_TP     0
SOA_Post_Lvl_Ind    0
Preferred_Class     0
Policies_Exposed    0
Death_Count         0
dtype: int64


In [15]:
# Numerical statistics
print('=== Numerical Feature Statistics ===')
print(df[NUMERICAL_FEATURES].describe())

=== Numerical Feature Statistics ===
       Attained_Age     Issue_Age      Duration
count  4.550104e+07  4.550104e+07  4.550104e+07
mean   5.294412e+01  4.026115e+01  1.368296e+01
std    1.764405e+01  1.674750e+01  1.044001e+01
min    0.000000e+00  0.000000e+00  1.000000e+00
25%    4.100000e+01  2.900000e+01  6.000000e+00
50%    5.300000e+01  4.000000e+01  1.200000e+01
75%    6.500000e+01  5.200000e+01  1.900000e+01
max    1.200000e+02  1.000000e+02  1.190000e+02


In [16]:
# Categorical distributions
print('=== Categorical Feature Distributions ===')
for col in CATEGORICAL_FEATURES:
    print(f'\n{col}:')
    vc = df[col].value_counts()
    if len(vc) > 10:
        print(vc.head(10))
        print(f'  ... ({len(vc)} unique values)')
    else:
        print(vc)

=== Categorical Feature Distributions ===

Sex:
Sex
M    24333152
F    21167884
Name: count, dtype: int64

Smoker_Status:
Smoker_Status
NS    32872436
S      8695643
U      3932957
Name: count, dtype: int64

Insurance_Plan:
Insurance_Plan
Term     20929192
Perm      7470281
UL        6075946
ULSG      4613219
VL        3120515
VLSG      2412344
Other      879539
Name: count, dtype: int64

Face_Amount_Band:
Face_Amount_Band
05: 100,000 - 249,999        10118882
06: 250,000 - 499,999         7396842
04: 50,000 - 99,999           6460460
07: 500,000 - 999,999         5471458
03: 25,000 - 49,999           4619883
08: 1,000,000 - 2,499,999     3919912
02: 10,000 - 24,999           3030835
01: 0 - 9,999                 2269246
09: 2,500,000 - 4,999,999     1204673
10: 5,000,000 - 9,999,999      676346
Name: count, dtype: int64
  ... (11 unique values)

Preferred_Class:
Preferred_Class
NA     14615884
2.0     7417733
1.0     7172721
2       4015772
1       3792389
3.0     3538797
3       1969

In [17]:
# Stage 2: Year distribution
print('=== Observation_Year Distribution (for Stage 2) ===')
print(df['Observation_Year'].value_counts().sort_index())

=== Observation_Year Distribution (for Stage 2) ===
Observation_Year
2012    4273432
2013    4608378
2014    4877434
2015    4998104
2016    5119495
2017    5293054
2018    8122436
2019    8208703
Name: count, dtype: int64


## 4. Save Cleaned Data

In [18]:
# Save as parquet (efficient for large datasets)
print(f'Saving cleaned data to {OUTPUT_PATH}...')
df.to_parquet(OUTPUT_PATH, index=False)

# Verify saved file
file_size = Path(OUTPUT_PATH).stat().st_size / 1e9
print(f'Saved! File size: {file_size:.2f} GB')

# Quick verification
df_verify = pd.read_parquet(OUTPUT_PATH)
print(f'Verification - Shape: {df_verify.shape}')
print('\nData cleaning complete!')

Saving cleaned data to ../data/ilec_cleaned.parquet...
Saved! File size: 0.29 GB
Verification - Shape: (45501036, 14)

Data cleaning complete!


## Summary

### Two-Stage Modeling Architecture

| Stage | Purpose | Features |
|-------|---------|----------|
| **Stage 1** | Baseline mortality prediction | 11 core features (no Year) |
| **Stage 2** | Residual analysis | Observation_Year |

### Stage 1 Features (11)
| Type | Count | Features |
|------|-------|----------|
| Numerical | 3 | Attained_Age, Issue_Age, Duration |
| Categorical | 8 | Sex, Smoker_Status, Insurance_Plan, Face_Amount_Band, Preferred_Class, SOA_Post_Lvl_Ind, SOA_Antp_Lvl_TP, SOA_Guar_Lvl_TP |

### Target & Weight
- **Target**: Death_Count
- **Weight**: Policies_Exposed

### Processing Applied
- Filled NA → 'NA' category for: Preferred_Class, SOA_Post_Lvl_Ind
- Converted categorical columns to category dtype
- Saved as parquet format

### Next Steps
1. Train Stage 1 model (LGBM Poisson)
2. Calculate residuals
3. Analyze residual patterns by Observation_Year